In [1]:
import sys
sys.path.append('../RecSysRep/')

In [2]:
import Basics.Load as ld
import scipy.sparse as sps

URM_all, ICM1, ICM2, ICM3, ICM4 = ld.getCOOs()
ICML = ld.getICMlength('5km')
ICM_weighted = ld.getICMselected('7')
ICM_gen = sps.hstack([ICM1, ICM2])
ICM_all = ld.getICMall()
# URM_train, URM_val = ld.getSplit(URM_train_val, 5678, 0.8)

In [3]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed=7777)
# URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
# evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13643 ( 0.1%) Users that have less than 1 test interactions


In [37]:
import os

ofp = "../models_temp/Similarity_Hybrid/seed7777/"

models_to_combine_best = {
                            'EASE': {"topK": None, "normalize_matrix": False, "l2_norm": 3749.2876352484},
                            'IALS_Hyb' : {"num_factors": 28, "epochs": 10, "confidence_scaling": "linear", "alpha": 0.43657990940994623, "epsilon": 0.35472063248578317, "reg": 0.0001698292271931609, "mw": 0.06122362507952762},
                            'RP3ICMnew': {'alpha': 1.029719677583138, 'beta': 1.0630164752134375, 'topK': 6964, 'normalize_similarity': True},
                            'RP3ICM' : {"topK": 2550, "alpha": 1.3058102610510849, "beta": 0.5150718337969987, "normalize_similarity": True, "implicit": True},
                            'IALS' :  {'num_factors': 34, 'epochs': 599, 'confidence_scaling': 'linear', 'alpha': 0.003519435539271083, 'epsilon': 0.09222402080721787, 'reg': 2.4127708108457617e-05},
                            'SLIMgensub': {"l1_ratio" : 0.025887359156206147, "topK": 2140, "alpha": 0.009567288586539689, "workers": 8, "mw": 1},
                            'SLIMBPR' : {"epochs": 440, "lambda_i": 0.007773815998802306, "lambda_j": 0.003342522366982381, "learning_rate": 0.010055161410725193, "topK": 4289, "random_seed": 1234, "sgd_mode": "sgd"},
                            'SLIMweig': {'l1_ratio': 0.0005247075138160404, 'topK': 4983, 'alpha': 0.06067400905430761, 'workers': 8, 'mw': 2.308619939318322},
                            'SLIMER': {'topK': 6000, 'l1_ratio': 0.0005495104968035837, 'alpha': 0.08007142704041009, 'workers': 8},
                            'P3alpha': {'topK': 4834, 'alpha': 1.764994849187595, 'normalize_similarity': True, 'implicit': True},
                            'RP3beta': {"topK": 1049, "alpha": 1.1626473723475605, "beta": 0.6765017195261293, "normalize_similarity": True, "implicit": True},
                            'icm_weighted': {"shrink": 4000, "topK": 985, "feature_weighting": "TF-IDF", "normalize": True},
                            'SLIM_all' : {'l1_ratio': 0.0001043005302985496, 'topK': 1135, 'alpha': 0.06322495726710943, 'workers': 8, 'mw': 1.2937257828415842},
                            'SLIM_all2' : {'topK': 4997, 'l1_ratio': 0.00036065147205858187, 'alpha': 0.03203200564199538, 'mw': 1.0785132020354402, 'workers': 8},
                            'UserKNN' : {"topK": 469, "similarity": "cosine", "shrink": 588, "normalize": True, "feature_weighting": "TF-IDF", "URM_bias": False},
                            'SLIM_BPR_Hyb' : {"epochs": 1443, "lambda_i": 8.900837513818856e-05, "lambda_j": 1.2615223007492727e-05, "learning_rate": 0.0037706733838839264, "topK": 6181, "random_seed": 1234, "sgd_mode": "sgd"},
                            'Neural': {"epochs": 270, "learning_rate": 0.00024353880350908526, "l2_reg": 0.003835663279264412, "dropout": 0.2398024286261501, "total_anneal_steps": 188362, "anneal_cap": 0.17528653915231018, "batch_size": 128, "encoding_size": 202, "next_layer_size_multiplier": 10, "max_n_hidden_layers": 1, "max_layer_size": 5000.0}
                         }


In [32]:
def model_init(recommender, name, args):
    path_name = ofp + name + ".zip"
    print(path_name)
    if os.path.exists(path_name):
        print("Model found!")
        recommender.load_model(ofp, name)
    else:
        print("Model does not exists, creating...")
        if not os.path.exists(ofp):
            print("Main folder does not exist, creating...")
            os.makedirs(ofp)
        recommender.fit(**args)
        recommender.save_model(ofp, name)

In [10]:
from Recommenders.NonPersonalizedRecommender import TopPop
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython_Hybrid
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIM_S_ElasticNetRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommenderICM
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender_Hybrid
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.Neural.MultVAERecommender import MultVAERecommender_OptimizerMask
from Recommenders.Neural.MultVAERecommender import MultVAERecommender

In [11]:
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from Recommenders.KNN.ItemKNNSimilarityHybridRecommenderNormal import ItemKNNSimilarityHybridRecommenderNormal
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridMultipleRecommender
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridTwoRecommender
from Recommenders.HybridScores.DifferentStructure import TwoDifferentModelRecommender

In [38]:
recommender1 = MultiThreadSLIM_SLIM_S_ElasticNetRecommender(URM_train, ICM_all)
recommender3 = EASE_R_Recommender(URM_train)

model_init(recommender1, 'SLIMall2', models_to_combine_best['SLIM_all2'])
model_init(recommender3, 'EASE', models_to_combine_best['EASE'])

../models_temp/Similarity_Hybrid/seed7777/SLIMall2.zip
Model found!
SLIMElasticNetRecommender: Loading model from file '../models_temp/Similarity_Hybrid/seed7777/SLIMall2'
SLIMElasticNetRecommender: Loading complete
../models_temp/Similarity_Hybrid/seed7777/EASE.zip
Model found!
EASE_R_Recommender: Loading model from file '../models_temp/Similarity_Hybrid/seed7777/EASE'
EASE_R_Recommender: Loading complete


In [13]:
recommender2 = MultVAERecommender_OptimizerMask(URM_train)
model_init(recommender2, 'Neural', models_to_combine_best['Neural'])

../models_temp/Similarity_Hybrid/seed7777/Neural.zip
Model does not exists, creating...
MultVAERecommender: Architecture: [202, 18059]
MultVAERecommender: Using default Temp folder './result_experiments/__Temp_MultVAERecommender_17060/'
MultVAERecommender: Saving model in file './result_experiments/__Temp_MultVAERecommender_17060/_best_model'
MultVAERecommender: Saving complete
MultVAERecommender: Epoch 1 of 270. Elapsed time 4.61 sec
MultVAERecommender: Epoch 2 of 270. Elapsed time 7.78 sec
MultVAERecommender: Epoch 3 of 270. Elapsed time 11.00 sec
MultVAERecommender: Epoch 4 of 270. Elapsed time 14.46 sec
MultVAERecommender: Epoch 5 of 270. Elapsed time 17.89 sec
MultVAERecommender: Epoch 6 of 270. Elapsed time 21.21 sec
MultVAERecommender: Epoch 7 of 270. Elapsed time 24.57 sec
MultVAERecommender: Epoch 8 of 270. Elapsed time 28.05 sec
MultVAERecommender: Epoch 9 of 270. Elapsed time 31.47 sec
MultVAERecommender: Epoch 10 of 270. Elapsed time 35.12 sec
MultVAERecommender: Epoch 11 o

MultVAERecommender: Epoch 133 of 270. Elapsed time 7.47 min
MultVAERecommender: Epoch 134 of 270. Elapsed time 7.53 min
MultVAERecommender: Epoch 135 of 270. Elapsed time 7.59 min
MultVAERecommender: Epoch 136 of 270. Elapsed time 7.65 min
MultVAERecommender: Epoch 137 of 270. Elapsed time 7.71 min
MultVAERecommender: Epoch 138 of 270. Elapsed time 7.76 min
MultVAERecommender: Epoch 139 of 270. Elapsed time 7.82 min
MultVAERecommender: Epoch 140 of 270. Elapsed time 7.88 min
MultVAERecommender: Epoch 141 of 270. Elapsed time 7.94 min
MultVAERecommender: Epoch 142 of 270. Elapsed time 8.00 min
MultVAERecommender: Epoch 143 of 270. Elapsed time 8.06 min
MultVAERecommender: Epoch 144 of 270. Elapsed time 8.12 min
MultVAERecommender: Epoch 145 of 270. Elapsed time 8.18 min
MultVAERecommender: Epoch 146 of 270. Elapsed time 8.24 min
MultVAERecommender: Epoch 147 of 270. Elapsed time 8.30 min
MultVAERecommender: Epoch 148 of 270. Elapsed time 8.37 min
MultVAERecommender: Epoch 149 of 270. El

MultVAERecommender: Epoch 269 of 270. Elapsed time 15.62 min
MultVAERecommender: Epoch 270 of 270. Elapsed time 15.68 min
MultVAERecommender: Saving model in file './result_experiments/__Temp_MultVAERecommender_17060/_best_model'
MultVAERecommender: Saving complete
MultVAERecommender: Terminating at epoch 270. Elapsed time 15.69 min
MultVAERecommender: Loading model from file './result_experiments/__Temp_MultVAERecommender_17060/_best_model'
INFO:tensorflow:Restoring parameters from ./result_experiments/__Temp_MultVAERecommender_17060/_best_model/session
MultVAERecommender: Loading complete
MultVAERecommender: Cleaning temporary files from './result_experiments/__Temp_MultVAERecommender_17060/'
MultVAERecommender: Saving model in file '../models_temp/Similarity_Hybrid/seed7777/Neural'
MultVAERecommender: Saving complete


EASE_R_Recommender: Fitting model... 
Similarity column 18059 (100.0%), 661.89 column/sec. Elapsed time 27.28 sec
EASE_R_Recommender: Fitting model... done in 3.70 min
EvaluatorHoldout: Processed 13643 (100.0%) in 42.96 sec. Users per second: 318
             MAP
cutoff          
10      0.244928
4000 : 0.24492767091444193
EASE_R_Recommender: Fitting model... 
Similarity column 18059 (100.0%), 630.12 column/sec. Elapsed time 28.66 sec
EASE_R_Recommender: Fitting model... done in 3.40 min
EvaluatorHoldout: Processed 13643 (100.0%) in 46.99 sec. Users per second: 290
             MAP
cutoff          
10      0.244682
4200 : 0.24468221100651824
EASE_R_Recommender: Fitting model... 


KeyboardInterrupt: 

In [22]:
ofp = "../models_temp/Similarity_Hybrid/New/"

URM_train_temp, URM_validation_temp = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed=1234)
recommender_temp = EASE_R_Recommender(URM_train_temp)

#model_init(recommender1, 'SLIMall', models_to_combine_best['SLIM_all'])
model_init(recommender_temp, 'EASE', models_to_combine_best['EASE'])

../models_temp/Similarity_Hybrid/New/EASE.zip
Model found!
EASE_R_Recommender: Loading model from file '../models_temp/Similarity_Hybrid/New/EASE'
EASE_R_Recommender: Loading complete


In [26]:
evaluator_validation_temp = EvaluatorHoldout(URM_validation_temp, cutoff_list=[10])
evaluator_validation_temp.evaluateRecommender(recommender_temp)

EvaluatorHoldout: Ignoring 13643 ( 0.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 13643 (100.0%) in 46.60 sec. Users per second: 293
             MAP
cutoff          
10      0.246991


AttributeError: 'tuple' object has no attribute 'loc'

In [28]:
recommender_temp.fit(topK = None, normalize_matrix = False, l2_norm = 3750)
evaluator_validation_temp.evaluateRecommender(recommender_temp)

EASE_R_Recommender: Fitting model... 
Similarity column 18059 (100.0%), 658.81 column/sec. Elapsed time 27.41 sec
EASE_R_Recommender: Fitting model... done in 3.62 min
EvaluatorHoldout: Processed 13643 (100.0%) in 45.78 sec. Users per second: 298
             MAP
cutoff          
10      0.247259


(             MAP
 cutoff          
 10      0.247259,
 'CUTOFF: 10 - MAP: 0.2472595, \n')

In [33]:
import numpy as np

def test_percentage_sim(recommender_a, recommender_b, evaluator, high1, low1, step):
    recommender = ItemKNNSimilarityHybridRecommenderNormal(URM_train, recommender_a.W_sparse, recommender_b.W_sparse)
    results = []
    alp_space = np.linspace(high1, low1, step, True)
    for alp in alp_space:
        recommender.fit(alp)
        r_d, _ = evaluator.evaluateRecommender(recommender)
        print(alp, ":", r_d.loc[10]['MAP'])
        results.append(r_d.loc[10]['MAP'])
    
    return alp_space, results

def test_percentage_sco(recommender_a, recommender_b, evaluator, high1, low1, step):
    recommender = ItemKNNScoresHybridTwoRecommender(URM_train, recommender_a, recommender_b)
    results = []
    alp_space = np.linspace(high1, low1, step, True)
    for alp in alp_space:
        recommender.fit(alp)
        r_d, _ = evaluator.evaluateRecommender(recommender)
        print(alp, ":", r_d.loc[10]['MAP'])
        results.append(r_d.loc[10]['MAP'])
    
    return alp_space, results

In [24]:
def rank_models(evaluator, recommenders):
    for r in recommenders:
        r_d, _ = evaluator.evaluateRecommender(r)
        print(r.RECOMMENDER_NAME, r_d.loc[10]['MAP'])

In [39]:
rank_models(evaluator_validation, [recommender1, recommender2, recommender3])

EvaluatorHoldout: Processed 13643 (100.0%) in 21.54 sec. Users per second: 633
             MAP
cutoff          
10      0.248679
SLIMElasticNetRecommender 0.24867923721566704
EvaluatorHoldout: Processed 13643 (100.0%) in 8.92 sec. Users per second: 1529
             MAP
cutoff          
10      0.226851
MultVAERecommender 0.22685124879436958
EvaluatorHoldout: Processed 13643 (100.0%) in 43.91 sec. Users per second: 311
             MAP
cutoff          
10      0.244937
EASE_R_Recommender 0.24493658879662814


In [43]:
from Recommenders.HybridScores.DifferentStructure import ThreeDifferentModelRecommender
import optuna

class Objective(object):
    def __init__(self, URM_train, recommender1, recommender2, recommender3, evaluator):
        # Hold this implementation specific arguments as the fields of the class.
        self.URM_train = URM_train
        self.evaluator = evaluator
        self.recommender_1 = recommender1
        self.recommender_2 = recommender2
        self.recommender_3 = recommender3

    def __call__(self, trial):
        # Calculate an objective value by using the extra arguments.
        w_1 = trial.suggest_uniform('w_1', 0, 1)
        w_2 = trial.suggest_uniform('w_2', 0, 1)
        w_3 = trial.suggest_uniform('w_3', 0, 1)

        recommender_final = ThreeDifferentModelRecommender(self.URM_train.tocsr(), self.recommender_1, self.recommender_2, self.recommender_3)
        recommender_final.fit(2, w_1, w_2, w_3)
        result_dict, _ = self.evaluator.evaluateRecommender(recommender_final)

        map_v = -result_dict.loc[10]['MAP']
        if map_v >= -0.245: # minimum acceptable map
            # Calculate the penalty.
            trial.report(map_v, 0)

            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

        return map_v

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(Objective(URM_train, recommender1, recommender2, recommender3, evaluator_validation), n_trials=500)

print(study.best_params)

[I 2022-01-06 18:41:33,356] A new study created in memory with name: no-name-9bc55395-1c02-4c23-b5fa-86b932df1f5d


CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.23794527150535047
MultVAERecommender with weight beta: 0.4550342301907402
EASE_R_Recommender with weight gamma: 0.30702049830390926
Norm type: 2


In [25]:
ofp = "../all_models/URM_all/"

recommender1a = MultiThreadSLIM_SLIM_S_ElasticNetRecommender(URM_all, ICM_all)
recommender2a = MultVAERecommender_OptimizerMask(URM_all)

model_init(recommender1a, 'SLIMall', models_to_combine_best['SLIM_all'])
model_init(recommender2a, 'Neural', models_to_combine_best['Neural'])

../all_models/URM_all/SLIMall.zip
Model found!
SLIMElasticNetRecommender: Loading model from file '../all_models/URM_all/SLIMall'
SLIMElasticNetRecommender: Loading complete
../all_models/URM_all/Neural.zip
Model does not exists, creating...
MultVAERecommender: Architecture: [202, 18059]
MultVAERecommender: Using default Temp folder './result_experiments/__Temp_MultVAERecommender_16360/'
MultVAERecommender: Saving model in file './result_experiments/__Temp_MultVAERecommender_16360/_best_model'
MultVAERecommender: Saving complete
MultVAERecommender: Epoch 1 of 270. Elapsed time 2.97 sec
MultVAERecommender: Epoch 2 of 270. Elapsed time 5.43 sec
MultVAERecommender: Epoch 3 of 270. Elapsed time 7.87 sec
MultVAERecommender: Epoch 4 of 270. Elapsed time 10.33 sec
MultVAERecommender: Epoch 5 of 270. Elapsed time 12.78 sec
MultVAERecommender: Epoch 6 of 270. Elapsed time 15.22 sec
MultVAERecommender: Epoch 7 of 270. Elapsed time 17.67 sec
MultVAERecommender: Epoch 8 of 270. Elapsed time 20.13 

MultVAERecommender: Epoch 131 of 270. Elapsed time 6.67 min
MultVAERecommender: Epoch 132 of 270. Elapsed time 6.73 min
MultVAERecommender: Epoch 133 of 270. Elapsed time 6.80 min
MultVAERecommender: Epoch 134 of 270. Elapsed time 6.86 min
MultVAERecommender: Epoch 135 of 270. Elapsed time 6.92 min
MultVAERecommender: Epoch 136 of 270. Elapsed time 6.98 min
MultVAERecommender: Epoch 137 of 270. Elapsed time 7.04 min
MultVAERecommender: Epoch 138 of 270. Elapsed time 7.10 min
MultVAERecommender: Epoch 139 of 270. Elapsed time 7.17 min
MultVAERecommender: Epoch 140 of 270. Elapsed time 7.23 min
MultVAERecommender: Epoch 141 of 270. Elapsed time 7.29 min
MultVAERecommender: Epoch 142 of 270. Elapsed time 7.35 min
MultVAERecommender: Epoch 143 of 270. Elapsed time 7.41 min
MultVAERecommender: Epoch 144 of 270. Elapsed time 7.48 min
MultVAERecommender: Epoch 145 of 270. Elapsed time 7.53 min
MultVAERecommender: Epoch 146 of 270. Elapsed time 7.59 min
MultVAERecommender: Epoch 147 of 270. El

MultVAERecommender: Epoch 267 of 270. Elapsed time 15.06 min
MultVAERecommender: Epoch 268 of 270. Elapsed time 15.13 min
MultVAERecommender: Epoch 269 of 270. Elapsed time 15.19 min
MultVAERecommender: Epoch 270 of 270. Elapsed time 15.26 min
MultVAERecommender: Saving model in file './result_experiments/__Temp_MultVAERecommender_16360/_best_model'
MultVAERecommender: Saving complete
MultVAERecommender: Terminating at epoch 270. Elapsed time 15.26 min
MultVAERecommender: Loading model from file './result_experiments/__Temp_MultVAERecommender_16360/_best_model'
INFO:tensorflow:Restoring parameters from ./result_experiments/__Temp_MultVAERecommender_16360/_best_model/session
MultVAERecommender: Loading complete
MultVAERecommender: Cleaning temporary files from './result_experiments/__Temp_MultVAERecommender_16360/'
MultVAERecommender: Saving model in file '../all_models/URM_all/Neural'
MultVAERecommender: Saving complete


In [37]:
recommender_final = ItemKNNScoresHybridTwoRecommender(URM_all, recommender1a, recommender2a)
recommender_final.fit(0.517796)

In [38]:
import pandas as pd
K=10

user_test_path = '../data/data_target_users_test.csv'
user_test_dataframe = pd.read_csv(filepath_or_buffer=user_test_path,
                                sep=",",
                                dtype={0:int})

subm_set = user_test_dataframe.to_numpy().T[0]


subm_res = {"user_id":[], "item_list":[]}

for user_id in subm_set:
    subm_res["user_id"].append(user_id)
    res = recommender_final.recommend(user_id, K)
    print("USER", user_id, "RECOMMENDED!")
    res = ' '.join(map(str, res))
    print(res)
    subm_res["item_list"].append(res)


# print(subm_res)

submission = pd.DataFrame.from_dict(subm_res)
# submission

from datetime import datetime
now = datetime.now() # current date and time


submission.to_csv('../subs/submission {:%Y_%m_%d %H_%M_%S}.csv'.format(now), index=False)

USER 0 RECOMMENDED!
3899 5433 8465 16365 16359 2243 6061 9130 10586 640
USER 1 RECOMMENDED!
3473 940 14400 16012 640 2118 16365 10118 9275 14358
USER 2 RECOMMENDED!
6177 1551 17022 17330 16365 15119 14229 4607 2846 5925
USER 3 RECOMMENDED!
8194 15551 16359 15082 960 8371 15507 16444 16012 3910
USER 4 RECOMMENDED!
8194 10586 6177 2152 10954 5299 9871 16494 6011 3473
USER 5 RECOMMENDED!
1846 9621 14396 16640 7867 11227 13098 13585 99 5312
USER 6 RECOMMENDED!
6214 985 16846 14358 14181 8207 7606 14868 13443 9621
USER 7 RECOMMENDED!
13963 15119 14031 3433 1885 4292 7751 17364 640 3768
USER 8 RECOMMENDED!
8635 3619 5605 2665 1259 10865 3768 5299 4853 4088
USER 9 RECOMMENDED!
5605 8207 1614 14181 13318 15117 3473 1551 16755 14078
USER 10 RECOMMENDED!
3899 14400 6214 14358 6975 4607 5273 16365 13229 2243
USER 11 RECOMMENDED!
14181 3473 11359 10865 249 6431 12390 10370 5577 13134
USER 12 RECOMMENDED!
3899 6724 3200 1137 1792 15778 14358 13963 6905 1259
USER 13 RECOMMENDED!
8635 6214 15119 1435

USER 121 RECOMMENDED!
640 13963 8635 12901 2665 10865 6011 1792 3433 13229
USER 122 RECOMMENDED!
2665 12901 14868 17416 15778 1792 10865 640 8207 4853
USER 123 RECOMMENDED!
13229 13134 16846 5273 3899 640 15778 1792 2772 1941
USER 124 RECOMMENDED!
3899 16946 7744 14181 1259 2665 1551 1137 6011 12186
USER 125 RECOMMENDED!
15750 6905 17936 13134 3899 5273 14181 10295 1836 3889
USER 126 RECOMMENDED!
3473 14181 11339 17748 6177 12944 7295 5433 3829 3942
USER 127 RECOMMENDED!
5141 15922 2687 6676 16759 10776 7003 17944 13235 3742
USER 128 RECOMMENDED!
6975 17364 9131 249 14181 10295 16946 1614 6589 16791
USER 129 RECOMMENDED!
6214 16365 10865 8011 12707 5273 640 15119 14181 11282
USER 130 RECOMMENDED!
17330 14181 11339 3829 1551 4204 16598 6670 14213 16846
USER 131 RECOMMENDED!
16946 7744 2772 16808 10295 17788 3768 3473 14358 4582
USER 132 RECOMMENDED!
3899 13134 1551 1259 12707 16365 8651 15119 1137 985
USER 133 RECOMMENDED!
6157 8011 13963 5433 8473 6177 7975 13295 7374 6011
USER 134 REC

USER 243 RECOMMENDED!
3899 4582 1619 15181 2886 7295 12901 8455 17936 6961
USER 244 RECOMMENDED!
13654 1208 3364 5403 13318 16038 4883 16681 17943 9776
USER 245 RECOMMENDED!
6214 14181 16365 1614 14358 249 10680 3942 1137 4173
USER 246 RECOMMENDED!
7716 13654 2376 11844 14266 9322 8635 5186 14181 1259
USER 247 RECOMMENDED!
9871 13638 1386 5403 5424 5974 12663 5605 17526 17388
USER 248 RECOMMENDED!
8502 10865 13725 12186 6011 15119 17788 17294 552 640
USER 249 RECOMMENDED!
1232 11430 5675 6214 2301 16365 3473 6016 3080 3899
USER 250 RECOMMENDED!
6214 14266 5273 6975 7716 16364 13229 10865 4582 1259
USER 251 RECOMMENDED!
8194 7867 14396 7744 16549 12764 985 18050 6351 6177
USER 252 RECOMMENDED!
6177 10865 14868 16364 16846 8207 13963 17330 12186 11359
USER 253 RECOMMENDED!
6177 11576 16365 3899 1232 17584 10562 8207 12380 202
USER 254 RECOMMENDED!
6177 17748 7412 11359 17356 9610 249 10562 5751 4078
USER 255 RECOMMENDED!
640 16365 955 15119 2746 17330 13971 17294 1619 4853
USER 256 RECOM

USER 375 RECOMMENDED!
9131 3768 419 17294 17364 2152 4607 15551 15398 4582
USER 376 RECOMMENDED!
7486 2748 14400 10295 1259 14917 16012 14485 4204 12356
USER 377 RECOMMENDED!
4582 15778 2886 6118 2152 5273 6589 3869 16791 14181
USER 378 RECOMMENDED!
973 15119 5300 3768 3899 12832 6652 985 17364 17140
USER 379 RECOMMENDED!
14485 17584 9744 12457 14093 2756 11038 13178 1208 10865
USER 380 RECOMMENDED!
7407 17553 17748 78 3899 12028 17330 10562 13134 12939
USER 381 RECOMMENDED!
3473 10865 2886 10062 14181 12752 7225 16162 16486 17135
USER 382 RECOMMENDED!
13410 15119 16846 13963 17830 7919 14358 4388 1792 2152
USER 383 RECOMMENDED!
3473 8635 13229 15119 6975 6011 6652 16791 2886 13134
USER 384 RECOMMENDED!
6157 7744 17715 11058 3198 6214 4119 10865 17622 17958
USER 385 RECOMMENDED!
3473 5925 17416 1259 6975 14868 4582 5424 8502 1619
USER 386 RECOMMENDED!
13963 8898 2243 7264 14449 13950 3473 8096 10316 8316
USER 387 RECOMMENDED!
9610 3899 3473 3200 5403 287 3889 13410 8194 16846
USER 388 

USER 485 RECOMMENDED!
12752 16640 6214 14868 10865 15868 7606 11504 15289 4582
USER 486 RECOMMENDED!
3768 14358 5324 2665 13410 6011 14181 5273 4204 1137
USER 487 RECOMMENDED!
3889 6214 5141 4078 17670 1885 17330 10865 1792 1137
USER 488 RECOMMENDED!
12457 17330 6214 13134 8326 1208 14868 7716 16486 13178
USER 489 RECOMMENDED!
6540 11303 202 17159 8316 14813 11576 1838 7089 5675
USER 490 RECOMMENDED!
14400 3899 241 6214 15750 1551 1259 16791 17022 17364
USER 491 RECOMMENDED!
12492 7309 3899 13318 1259 7429 12311 3473 3942 17943
USER 492 RECOMMENDED!
8898 13950 1838 2665 3204 13134 840 562 8747 5675
USER 493 RECOMMENDED!
3473 7716 13654 8651 103 9776 14868 1551 1208 4088
USER 494 RECOMMENDED!
7224 17330 6782 249 17135 13914 13134 17526 1941 8747
USER 495 RECOMMENDED!
15119 11339 17584 14358 6905 1941 640 8326 8247 10865
USER 496 RECOMMENDED!
13134 7716 8011 14358 16365 13229 2182 5925 6589 5460
USER 497 RECOMMENDED!
3473 8635 16364 5273 3899 14181 9441 13725 1551 2846
USER 498 RECOMMEND

...

USER 13110 RECOMMENDED!
3899 7264 10562 8635 7486 1941 16365 12492 7257 13318
USER 13111 RECOMMENDED!
6214 13134 3899 17330 17936 13318 10865 940 3768 12663
USER 13112 RECOMMENDED!
6975 2665 16486 16791 3619 8635 17330 13410 12943 11359
USER 13113 RECOMMENDED!
7099 3473 8635 6724 10865 14627 15778 6306 17411 2665
USER 13114 RECOMMENDED!
9621 15289 12530 3910 8371 3899 13295 13001 17330 12794
USER 13115 RECOMMENDED!
3200 1941 17936 13134 8326 9871 6905 17748 4204 3942
USER 13116 RECOMMENDED!
3200 1614 6724 1941 1259 15469 9441 5403 6905 12530
USER 13117 RECOMMENDED!
6177 17416 17294 13410 15119 16624 16846 2152 17330 17788
USER 13118 RECOMMENDED!
6177 3473 12901 1515 12390 9527 5324 12752 10865 1792
USER 13119 RECOMMENDED!
17330 14358 2796 14181 6975 11339 552 11491 7744 6670
USER 13120 RECOMMENDED!
3899 1581 14396 8635 15119 8567 15868 6214 5972 17330
USER 13121 RECOMMENDED!
2545 12752 9131 2772 6214 16364 2552 3473 8194 3899
USER 13122 RECOMMENDED!
3473 15181 640 5273 17330 5403 10865

USER 13241 RECOMMENDED!
6177 3899 16808 12186 13387 1137 16486 17666 12943 3619
USER 13242 RECOMMENDED!
5403 3200 1551 8651 15119 6214 1259 3942 1137 14536
USER 13243 RECOMMENDED!
17830 15187 3130 6652 13229 3579 13410 13178 15880 6214
USER 13244 RECOMMENDED!
13229 13963 5403 2665 14868 6652 16364 3768 13134 16791
USER 13245 RECOMMENDED!
17520 5273 12457 3899 3473 1941 8635 11562 13178 3942
USER 13246 RECOMMENDED!
3899 14358 4292 7374 11430 5925 12792 15583 10954 6975
USER 13247 RECOMMENDED!
13914 6177 16012 15778 16364 8037 12378 13465 16353 17526
USER 13248 RECOMMENDED!
15119 2665 8207 7867 17462 3691 4582 13410 640 1551
USER 13249 RECOMMENDED!
6177 14396 14358 3768 17584 11227 4582 12901 15778 14181
USER 13250 RECOMMENDED!
12752 16701 16365 8207 2846 3473 8911 4919 6011 5178
USER 13251 RECOMMENDED!
5433 16028 960 6148 14952 3913 12794 2748 12530 9871
USER 13252 RECOMMENDED!
6177 12457 1941 8502 10865 14627 13229 13914 803 9871
USER 13253 RECOMMENDED!
3473 6214 8567 14400 5974 6177 1

USER 13355 RECOMMENDED!
16846 15181 5299 12901 955 2746 2886 17961 525 6782
USER 13356 RECOMMENDED!
5299 12901 6177 13410 2152 8207 13229 985 17330 3955
USER 13357 RECOMMENDED!
17748 3889 14400 5751 15778 14485 4093 8635 12356 2456
USER 13358 RECOMMENDED!
3899 15119 3768 8635 13032 13410 13134 12186 14181 6975
USER 13359 RECOMMENDED!
7867 16846 13295 7975 17935 4607 2886 13410 5273 12752
USER 13360 RECOMMENDED!
1846 6214 6975 5273 16624 640 1792 388 17330 8194
USER 13361 RECOMMENDED!
14781 6177 2772 14181 3829 15778 17022 16486 13032 3942
USER 13362 RECOMMENDED!
8635 5273 1208 16846 15119 1551 10370 2665 17330 14940
USER 13363 RECOMMENDED!
16846 14181 15119 6975 16624 14358 6011 17416 640 2665
USER 13364 RECOMMENDED!
1208 10370 3829 6214 6670 11491 1259 14868 16365 17943
USER 13365 RECOMMENDED!
2376 1259 5403 15778 1614 5742 2820 17135 12457 249
USER 13366 RECOMMENDED!
16946 15289 6417 6670 5403 3473 10865 13295 1603 14868
USER 13367 RECOMMENDED!
17936 9610 12663 13914 8502 10370 15750

USER 13482 RECOMMENDED!
16846 6214 3473 13229 2665 9131 2182 5299 12186 10954
USER 13483 RECOMMENDED!
6177 14181 14400 5742 3473 6905 14095 10370 17356 6214
USER 13484 RECOMMENDED!
16919 2886 3768 16365 13032 5324 14536 16846 4088 13229
USER 13485 RECOMMENDED!
9875 8635 3768 11391 8584 10908 16486 10865 7744 3232
USER 13486 RECOMMENDED!
5273 9322 1614 6306 2665 9527 2748 15119 7973 15469
USER 13487 RECOMMENDED!
15119 5273 17135 12901 3942 13914 14400 1792 8207 2756
USER 13488 RECOMMENDED!
17294 6011 17330 2665 1619 15778 14181 12186 10865 13410
USER 13489 RECOMMENDED!
6214 4078 16332 2886 1044 13229 4582 16494 11339 14873
USER 13490 RECOMMENDED!
5300 5324 15119 13134 1619 13229 10865 14181 1551 12901
USER 13491 RECOMMENDED!
7744 16365 6975 15119 2076 6011 4199 13585 11343 6351
USER 13492 RECOMMENDED!
13134 16486 3473 1208 9871 10865 6011 2886 8651 17294
USER 13493 RECOMMENDED!
8011 3889 8635 5974 1614 3955 8247 16549 1515 13801
USER 13494 RECOMMENDED!
3899 1259 12390 13178 4204 16486 5

USER 13590 RECOMMENDED!
17584 3899 7257 10562 8635 8326 14181 11576 1551 5925
USER 13591 RECOMMENDED!
8635 1137 11359 3829 17330 1792 6011 4292 16791 10370
USER 13592 RECOMMENDED!
14396 16444 13295 14159 9898 8194 7867 15712 14444 6976
USER 13593 RECOMMENDED!
6011 5424 16012 8473 3198 2807 5714 6157 14917 3889
USER 13594 RECOMMENDED!
8011 6011 1619 12901 2772 14226 53 15082 5433 14159
USER 13595 RECOMMENDED!
6177 2243 6975 8207 985 15778 13229 249 5273 3768
USER 13596 RECOMMENDED!
6782 8502 3899 1137 1941 1551 4088 12663 16353 10865
USER 13597 RECOMMENDED!
3200 3899 11339 7486 8326 7716 4088 1259 13914 17748
USER 13598 RECOMMENDED!
13134 1137 4204 13229 8635 7309 16070 17022 6177 14400
USER 13599 RECOMMENDED!
6724 8635 17411 17330 4853 14358 5273 5403 1581 16365
USER 13600 RECOMMENDED!
6177 6652 13963 16365 4292 6011 1102 17364 2152 13134
USER 13601 RECOMMENDED!
10865 1137 9871 5925 4204 17022 14213 5962 14181 16846
USER 13602 RECOMMENDED!
17356 13914 8635 4078 15778 3666 17022 1581 75